# Extended Real Quantum Amplitude Estimation (RQAE) module

The present notebook explain how the **extended Real Quantum Amplitude Estimation** (*eRQAE*) is implemented into the **QQuantLib** library.

The **eRQAE** algorithm is a modification of the **Real Quantum Amplitude Estimation** (RQAE) algorithms where the amplification steps can be guided by the user by providing a scheduler. The **eRQAE** keeps the benefits of the original **RQAE** (estimating the amplitude and its corresponding sign) by allows to the user guide the amplification steps to get a better perfomance (better relation between desired error and calls to the oracle) than the  **RQAE** one.

The **eRQAE** algorithm was implemented as a Python class (*eRQAE*) inside the *extended_real_quantum_ae* module within the package *AE* of the library *QQuantLib* (**QQuantLib/AE/extended_real_quantum_ae.py**).

The **eRQAE** algorithm is based in the **RQAE** algorithm:

* Manzano, A., Musso, D. & Leitao, Á. Real quantum amplitude estimation. EPJ Quantum Technol. 10, 2 (2023) (https://epjquantumtechnology.springeropen.com/articles/10.1140/epjqt/s40507-023-00159-0#citeas)


In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. QPU = [qlmass, python, c]
from QQuantLib.qpu.get_qpu import get_qpu
QPU = ["qlmass", "python", "c"]
linalg_qpu = get_qpu(QPU[2])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results
from QQuantLib.utils.utils import bitfield_to_int

## 1. Oracle generation

Before doing any amplitude estimation we want to load some data into the quantum circuit, as this step is only auxiliary to see how the algorithm works, we are just going to load a discrete probability distribution. In this case, we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$


In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function *load_probability* from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
oracle = load_probability(probability)

In [ ]:
%qatdisplay oracle --svg

For more information about loading data into the quantum circuit see the notebook *01_DataLoading_Module_Use*.

## 2. eRQAE class

### 2.1 The Amplitude Estimation Problem

The **eRQAE** algorithm is a modification of the **RQAE** one so it solves the **amplitude estimation** problem when a little variation is added.  In this case, given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = a|\Psi_0\rangle +\sqrt{1-a^2}|\Psi_1\rangle, \tag{1}$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, *we want to estimate the real parameter $a$ (so $a$ can take values in the domain $[-1,1]$)*

**BE AWARE** 

In Notebooks: *03_Maximum_Likelihood_Amplitude_Estimation_Class.ipynb*, *04_Classical_Phase_Estimation_Class.ipynb*, *05_Iterative_Quantum_Phase_Estimation_Class.ipynb*, *06_Iterative_Quantum_Amplitude_Estimation_class.ipynb* we want to estimate $\sqrt{a}$ meanwhile in this new problem we want to estimate $a$


### 2.2 eRQAE algorithm output

Given an error $\epsilon$ and a confident interval $\gamma$, the **eRQAE** algorithm allows to estimate the $a$, from the amplitude estimation problem presented in *Section 2.1*, satisfying:

$$P\big[a \in [a_l, a_u]\big] \gt 1-\gamma$$

and

$$\frac{a_u-a_l}{2} \leq \epsilon$$


To show how our **RQAE** class works, we will define the following amplitude estimation problem:

$$|\Psi\rangle = \mathcal{A}|0\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right] \tag{2}$$

So comparing (2) with (1):

$$a|\Psi_0\rangle =  \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle$$

and 

$$\sqrt{1-a^2}|\Psi_1\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

The target state, in this case, is $|1\rangle$. Its binary representation is $001$. This has to be passed to the target variable as a list. Moreover, we have to provide the list of qubits where we are acting, in this case is just $[0,1,2]$, the whole register.

### 2.3 Creating object from RQAE class

We have implemented and python class called **eRQAE** into the **QQuantLib/AE/extended_real_quantum_ae** module that allows us to use the **eRQAE** algorithm.

For creating the **eRQAE** class the conventions used in **MLAE class** from **QQuantLib/AE/maximum_likelihood_ae.py** module should be followed: 

We have some mandatory inputs:

1. Oracle: QLM AbstractGate or QRoutine with the implementation of the Oracle for creating the Grover operator.
2. target: this is the marked state in binary representation as a Python list. 
3. index: list of the qubits affected by the Grover operator. 

And some optional inputs, used for algorithm configuration, that can be given as a Python dictionary:
* qpu: QLM solver that will be used
* epsilon ($\epsilon$): the precision. Ensures that the width of the interval is (see Section 2.2), at most, $2\epsilon$ (default: 0.01).
* gamma ($\gamma$): the accuracy. Ensures that the probability of $a$ not laying within the given interval (see Section 2.2) is, at most, $\gamma$ (default: 0.05).
* mcz_qlm: for using QLM multi-controlled Z gate (True, default) or using multiplexor implementation (False)
* erqae_schedule: Python dictionary that allows to build the amplification schedule of the algorithm

The new part of the **eRQAE** algorihtm is that the user can guide the amplification schedule of the algorightm . 

We explain how to fix this in the following sub section

#### The **eRQAE** schedule

To guide the amplification steps during the **eRQAE** algorithm execution the user can provided to the algorithm the two following inputs:

* **Amplification list**: is a list where the user proposes amplifications for each step of the algorithm. This is at each step the user propose what is the number of times the corresponding **Grover** operator will be applied ($k_i$)
* **Confidence list**: is a list where the user proposes the confidence level for each step of the algorithm. 

For example the user can want a scheduler that increases the amplification in an exponential way meanwhile the confidence can only 

In [ ]:
from QQuantLib.AE.extended_real_quantum_ae import select_schedule

In [ ]:
schedule_exp_cons = {
    "type": "exp_const",
    "ratio_slope_k": 2,
    "ratio_slope_gamma": None
}

schedule_exp_exp = {
    "type": "exp_exp",
    "ratio_slope_k": 2,
    "ratio_slope_gamma": 2
}

schedule_lin_lin = {
    "type": "linear_linear",
    "ratio_slope_k": 2,
    "ratio_slope_gamma": 2
}

In [ ]:
epsilon = 0.01
gamma = 0.05

In [ ]:
list_k, list_gamma = select_schedule(schedule_exp_cons, epsilon, gamma)
fig, ax1 = plt.subplots()
ax1.set_title('Amplitude: Exponential. Confidence: Constant')
ax1.plot(list_k, '-o')
ax2 = ax1.twinx()
ax2.plot(list_gamma, '-o', color="r")
ax1.set_ylabel('Amplification schedule')
ax2.set_ylabel('Confidence schedule')
fig.legend(["Amplification schedule", "Confidence schedule"],
           loc='upper left', bbox_to_anchor=(0.2, 0.8))


In [ ]:
list_k, list_gamma = select_schedule(schedule_exp_exp, epsilon, gamma)

fig, ax1 = plt.subplots()
ax1.set_title('Amplitude: Exponential. Confidence: Exponential')
ax1.plot(list_k, '-o')
ax2 = ax1.twinx()
ax2.plot(list_gamma, '-o', color="r")
ax1.set_ylabel('Amplification schedule')
ax2.set_ylabel('Confidence schedule')
fig.legend(["Amplification schedule", "Confidence schedule"],
           loc='upper left', bbox_to_anchor=(0.2, 0.8))


In [ ]:
list_k, list_gamma = select_schedule(schedule_lin_lin, epsilon, gamma)

fig, ax1 = plt.subplots()
ax1.plot(list_k, '-o')
ax1.set_title('Amplitude: Linear. Confidence: Linear')
ax2 = ax1.twinx()
ax2.plot(list_gamma, '-o', color="r")
ax1.set_ylabel('Amplification schedule')
ax2.set_ylabel('Confidence schedule')
fig.legend(["Amplification schedule", "Confidence schedule"],
           loc='upper left', bbox_to_anchor=(0.2, 0.8))


## OPA

In [ ]:
#import the class
from QQuantLib.AE.extended_real_quantum_ae import eRQAE

In [ ]:
target = [0,0,1]
index = [0,1,2]

#We want to estimate the probability of target.
#In the RQAE solution the $a$ is codified as an amplitude not as a probability
a_real = np.sqrt(probability[bitfield_to_int(target)])

print('We want to estimate: ', a_real)



rqae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True,
    "epsilon" : 0.001,
    "gamma": 0.05,
    'erqae_schedule': schedule_exp_cons
    
}

e_rqae = eRQAE(oracle,target = [0,0,1],index = [0,1,2], **rqae_dict)

In [ ]:
print("epsilon: ", e_rqae.epsilon)
print("gamma: ", e_rqae.gamma)
print("erqae_schedule: ", e_rqae.erqae_schedule)
print("schedule_k: ", e_rqae.schedule_k)
print("schedule_gamma: ", e_rqae.schedule_gamma)

### 2.4 The *erqae* method

To execute the complete algorithm using the **eRQAE** class the *erqae* method can be used. 

 This method has the following inputs:
* epsilon ($\epsilon$): error in the estimation of $a$ (default: 0.01).
* gamma ($\gamma$): confidence interval for the $a$ estimation (default: 0.05).
* schedule_k : list with the amplification schedule
* schedule_gamma : list with the confidence schedule

This method returns the limits for the $a$ estimation: $(a_{\min},a_{\max})$

#### Schedule: amplification exponential confidence constant

In [ ]:
schedule_exp_cons = {
    "type": "exp_const",
    "ratio_slope_k": 2,
    "ratio_slope_gamma": None
}
list_k, list_gamma = select_schedule(
    schedule_exp_cons, rqae_dict["epsilon"], rqae_dict["gamma"])
bounds = e_rqae.erqae(
    epsilon=rqae_dict["epsilon"], 
    gamma=rqae_dict["gamma"],
    schedule_k=list_k,
    schedule_gamma=list_gamma
)

In [ ]:
print('Bounds for a: [a_l, a_u] = [{}, {}]'.format(bounds[0], bounds[1]))
a_estimated = (bounds[0]+bounds[1])/2.0
print('a_estimated: ', a_estimated)
print('a real: ', a_real)
print('|a_l-a_estimated| = ', np.abs(a_estimated-a_real))
print('Error estimation wanted: ', 0.01)

In [ ]:
if (a_real>=bounds[0])&(a_real<=bounds[1]):
    print("Correct")
else:
    print("Incorrect")

#### Schedule: amplification exponential confidence exponential

In [ ]:
schedule_exp_exp = {
    "type": "exp_exp",
    "ratio_slope_k": 2,
    "ratio_slope_gamma": 2
}
list_k, list_gamma = select_schedule(
    schedule_exp_exp, rqae_dict["epsilon"], rqae_dict["gamma"])
print(list_k)
print(list_gamma)
bounds = e_rqae.erqae(
    epsilon=rqae_dict["epsilon"], 
    gamma=rqae_dict["gamma"],
    schedule_k=list_k,
    schedule_gamma=list_gamma
)

In [ ]:
print('Bounds for a: [a_l, a_u] = [{}, {}]'.format(bounds[0], bounds[1]))
a_estimated = (bounds[0]+bounds[1])/2.0
print('a_estimated: ', a_estimated)
print('a real: ', a_real)
print('|a_l-a_estimated| = ', np.abs(a_estimated-a_real))
print('Error estimation wanted: ', 0.01)

#### Schedule: amplification linear confidence linear

In [ ]:
schedule_lin_lin = {
    "type": "linear_linear",
    "ratio_slope_k": 2,
    "ratio_slope_gamma": 2
}
list_k, list_gamma = select_schedule(
    schedule_lin_lin, rqae_dict["epsilon"], rqae_dict["gamma"])
print(list_k)
print(list_gamma)
bounds = e_rqae.erqae(
    epsilon=rqae_dict["epsilon"], 
    gamma=rqae_dict["gamma"],
    schedule_k=list_k,
    schedule_gamma=list_gamma
)

In [ ]:
print('Bounds for a: [a_l, a_u] = [{}, {}]'.format(bounds[0], bounds[1]))
a_estimated = (bounds[0]+bounds[1])/2.0
print('a_estimated: ', a_estimated)
print('a real: ', a_real)
print('|a_l-a_estimated| = ', np.abs(a_estimated-a_real))
print('Error estimation wanted: ', 0.01)

### 2.6 The *run* method

Finally, a *run* method for direct implementation of the **eRQAE** algorithm was implemented. In this case, the user can configure all the properties of the **eRQAE** class and the *run* method will execute the method using the fixed attributes of the class. Finally, the method returns the estimation of $a=\frac{a_u+a_l}{2}$. Additionally, the *run* method populates the following class attributes:

* *ae_l*: the lower limit for a $a_l$.
* *ae_u*: the upper limit for a $a_u$.
* *ae*: the amplitude estimation parameter calculated as: $a=\frac{a_u+a_l}{2}$
* *run_time*: elapsed time for a complete execution of the **run** method.


#### Schedule: amplification exponential confidence constant

In [ ]:
#First we create the class
target = [0,0,1]
index = [0,1,2]

#We want to estimate the probability of target.
#In the RQAE solution the $a$ is codified as an amplitude not as a probability
a_real = np.sqrt(probability[bitfield_to_int(target)])

print('We want to estimate: ', a_real)

epsilon = 0.001
gamma = 0.01
schedule_exp_cons = {
    "type": "exp_const",
    "ratio_slope_k": 2,
    "ratio_slope_gamma": None
}


rqae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True,
    "epsilon" : epsilon,
    "gamma": gamma,
    'erqae_schedule': schedule_exp_cons
    
}

e_rqae = eRQAE(oracle,target = [0,0,1],index = [0,1,2], **rqae_dict)
print(e_rqae.schedule_k)
print(e_rqae.schedule_gamma)

In [ ]:
a_estimated = e_rqae.run()

In [ ]:
print('a_estimated: ', a_estimated)
print('Real Value of a: ', a_real)
print('Bounds for a: [rqae.a_l, rqae.a_u] = [{}, {}]'.format(
    e_rqae.ae_l, e_rqae.ae_u))
print('Estimated a: rqae.a= ', e_rqae.ae)
print('|a_l-a_estimated| = ', np.abs(a_real-e_rqae.ae))
print('Error estimation wanted: ', e_rqae.epsilon)

In [ ]:
print("Elapsed time of a run method: ", e_rqae.run_time)

When the *run* method is executed following class attributes are populated:

* *circuit_statistics*: python dictionary with the statistics of each circuit used during the algorithm execution. Each key of the dictionary corresponds with a $k$ application of the Grover-like operator used and its associated value is a Python dictionary with the complete statistical information of the circuit created for each $k$ value.
* *schedule_pdf*: pandas DataFrame with the complete schedule used in the algorithm execution. The schedule lists the number of applications Grover-like applications and the number of shots used for measurements.
* *oracle_calls*: number of total oracle calls for a complete execution of the algorithm.
* *max_oracle_depth*: maximum number of applications of the oracle for the complete execution of the algorithm.

In [ ]:
e_rqae.circuit_statistics

In [ ]:
e_rqae.schedule_pdf

In [ ]:
#Total number of oracle calls
print("The total number of the oracle calls was: {}".format(
    e_rqae.oracle_calls))

In [ ]:
#Number of maximum oracle applications
e_rqae.max_oracle_depth

In [ ]:
e_rqae.quantum_time

In [ ]:
e_rqae.run_time

#### Schedule: amplification exponential confidence exponential

In [ ]:
#First we create the class
target = [0,0,1]
index = [0,1,2]

#We want to estimate the probability of target.
#In the RQAE solution the $a$ is codified as an amplitude not as a probability
a_real = np.sqrt(probability[bitfield_to_int(target)])

print('We want to estimate: ', a_real)

epsilon = 0.001
gamma = 0.01
schedule_exp_exp = {
    "type": "exp_exp",
    "ratio_slope_k": 2,
    "ratio_slope_gamma": 2
}


rqae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True,
    "epsilon" : epsilon,
    "gamma": gamma,
    'erqae_schedule': schedule_exp_exp
    
}

e_rqae = eRQAE(oracle,target = [0,0,1],index = [0,1,2], **rqae_dict)
print(e_rqae.schedule_k)
print(e_rqae.schedule_gamma)

In [ ]:
a_estimated = e_rqae.run()

In [ ]:
print('a_estimated: ', a_estimated)
print('Real Value of a: ', a_real)
print('Bounds for a: [rqae.a_l, rqae.a_u] = [{}, {}]'.format(
    e_rqae.ae_l, e_rqae.ae_u))
print('Estimated a: rqae.a= ', e_rqae.ae)
print('|a_l-a_estimated| = ', np.abs(a_real-e_rqae.ae))
print('Error estimation wanted: ', e_rqae.epsilon)
#Total number of oracle calls
print("The total number of the oracle calls was: {}".format(
    e_rqae.oracle_calls))

#### Schedule: amplification linear confidence linear

In [ ]:
#First we create the class
target = [0,0,1]
index = [0,1,2]

#We want to estimate the probability of target.
#In the RQAE solution the $a$ is codified as an amplitude not as a probability
a_real = np.sqrt(probability[bitfield_to_int(target)])

print('We want to estimate: ', a_real)

epsilon = 0.001
gamma = 0.01
schedule_lin_lin = {
    "type": "linear_linear",
    "ratio_slope_k": 2,
    "ratio_slope_gamma": 2
}


rqae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True,
    "epsilon" : epsilon,
    "gamma": gamma,
    'erqae_schedule': schedule_lin_lin
    
}

e_rqae = eRQAE(oracle,target = [0,0,1],index = [0,1,2], **rqae_dict)
print(e_rqae.schedule_k)
print(e_rqae.schedule_gamma)

In [ ]:
a_estimated = e_rqae.run()

In [ ]:
print('a_estimated: ', a_estimated)
print('Real Value of a: ', a_real)
print('Bounds for a: [rqae.a_l, rqae.a_u] = [{}, {}]'.format(
    e_rqae.ae_l, e_rqae.ae_u))
print('Estimated a: rqae.a= ', e_rqae.ae)
print('|a_l-a_estimated| = ', np.abs(a_real-e_rqae.ae))
print('Error estimation wanted: ', e_rqae.epsilon)
#Total number of oracle calls
print("The total number of the oracle calls was: {}".format(
    e_rqae.oracle_calls))

In [ ]:
e_rqae.schedule_pdf

## 3. Compatibility with AE class

In [ ]:
from QQuantLib.AE.ae_class import AE

In [ ]:

target = [0,0,1]
index = [0,1,2]

a_real = np.sqrt(probability[bitfield_to_int(target)])

print('We want to estimate: ', a_real)
ae_dict = {
    #QPU
    'qpu': linalg_qpu,
    #Multi controlat decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': 100,
    
    #MLAE
    'schedule': [],
    'delta' : 1.0e-6,
    'ns' : 10000,
    
    #CQPEAE
    'auxiliar_qbits_number': 10,
    #IQPEAE
    'cbits_number': 6,
    #IQAE & RQAQE
    'epsilon': 0.001,
    #IQAE
    'alpha': 0.05,
    #RQAE
    'gamma': 0.05,
    'q': 1.2,
    #eRQAE
    'erqae_schedule': {
        "type": "linear_linear",
        "ratio_slope_k": 2, 
        "ratio_slope_gamma": 2
    }
}


In [ ]:
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)
ae_object.ae_type = 'eRQAE'

In [ ]:
ae_object.run()

In [ ]:
print('Real Value of a: ', a_real)
print('a_estimated: ', ae_object.ae_pdf["ae"])

In [ ]:
ae_object.schedule_pdf

In [ ]:
ae_object.oracle_calls

## 4. Compatibility with q_integration

In [ ]:
from QQuantLib.finance.quantum_integration import q_solve_integral

In [ ]:
#Negative Riemann sum!!

a = np.pi

b = np.pi + np.pi / 4.0

#n will define the maximum numbers of our domain
n = 5
x = np.linspace(a, b, 2 ** n)

#function definition
f_x = np.sin(x)
#function normalisation
f_x_normalisation = np.max(np.abs(f_x)) + 1e-8
norm_f_x = f_x / f_x_normalisation

#probability definition
p_x = x
#probability normalisation
p_x_normalisation = np.sum(p_x) + 1e-8
norm_p_x = p_x / p_x_normalisation

#Desired Integral
riemman = np.sum(p_x * f_x)

In [ ]:
riemman

In [ ]:
#Testing normalised conditions!

print('p(x) condition: {}'.format(np.sum(norm_p_x) <= 1))
print('f(x) condition: {}'.format(np.max(np.abs(norm_f_x)) <= 1))

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(x, norm_p_x, label='p(x)')
ax1.set_ylabel('p(x)', color = 'b')
ax1.tick_params(axis='y', labelcolor='b')
ax2 = ax1.twinx()
ax2.plot(x, norm_f_x, color='r', label = 'f(x)')
ax2.set_ylabel('f(x)', color='r')
ax2.tick_params(axis='y', labelcolor='r')
fig.legend(['p(x)', 'f(x)'])

In [ ]:
ae_dict = {
    'qpu': linalg_qpu,
    #Multi controlat decomposition
    'mcz_qlm': False,   
    #For encoding class
    "multiplexor": True,   
    #eRQAE
    'erqae_schedule': {
        "type": "linear_linear",
        "ratio_slope_k": 2, 
        "ratio_slope_gamma": 2
    },
    'gamma': 0.05,    
    'epsilon': 0.001,    
}
ae_dict.update({
    "array_function":norm_f_x,
    "array_probability": norm_p_x,
})
ae_dict.update({"ae_type" : "eRQAE"})
ae_dict.update({"encoding" : 2})

In [ ]:
%%time
erqae_solution, erqae_object = q_solve_integral(**ae_dict)
erqae_rieman = erqae_solution*f_x_normalisation*p_x_normalisation
print("eRQAE Riemann :{}. Riemann: {}".format(
    erqae_rieman['ae'].iloc[0], riemman))
print("Absolute Error: {}".format(
    np.abs(erqae_rieman['ae'].iloc[0] - riemman)))